## setup libraries

In [1]:
#library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"
!pip install gdelt

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelea

In [2]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

## fetch gdelt data for a given date range

Note: this example only fetches the events table. The gdelt python API also can pull from the mentions and gkg tables. See https://linwoodc3.github.io/gdeltPyR/ for full API details



In [3]:
# these remove files from previous runs
!rm -rf articles
!rm *.csv

rm: cannot remove '*.csv': No such file or directory


In [4]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()


# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)

def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          d = gd.Search(date, table='gkg',coverage=False) #not updata at 15mins
          d.to_csv(filename,encoding='utf-8',index=False)
          return filename
    except:
        print("Error occurred")

# test gkg table
test_data_before = [get_filename(x) for x in pd.date_range('2019 Nov 27', '2019 Dec 28')]
dbg(test_data_before)

test_data_after = [get_filename(x) for x in pd.date_range('2019 Dec 29', '2020 Jan 30')]
dbg(test_data_after)
#downloading
results_1 = list(e.map(intofile, test_data_before))
results_2 = list(e.map(intofile, test_data_after))
#fix error bug
results_1_fix_bug = []
results_2_fix_bug = []

for i in results_1:
  if i != None:
    results_1_fix_bug.append(i)
results_1_fix_bug

for i in results_2:
  if i != None:
    results_2_fix_bug.append(i)
results_2_fix_bug

dbg(results_1_fix_bug)
dbg(results_2_fix_bug)


['20191127_gdeltdata.csv', '20191128_gdeltdata.csv', '20191129_gdeltdata.csv', '20191130_gdeltdata.csv', '20191201_gdeltdata.csv', '20191202_gdeltdata.csv', '20191203_gdeltdata.csv', '20191204_gdeltdata.csv', '20191205_gdeltdata.csv', '20191206_gdeltdata.csv', '20191207_gdeltdata.csv', '20191208_gdeltdata.csv', '20191209_gdeltdata.csv', '20191210_gdeltdata.csv', '20191211_gdeltdata.csv', '20191212_gdeltdata.csv', '20191213_gdeltdata.csv', '20191214_gdeltdata.csv', '20191215_gdeltdata.csv', '20191216_gdeltdata.csv', '20191217_gdeltdata.csv', '20191218_gdeltdata.csv', '20191219_gdeltdata.csv', '20191220_gdeltdata.csv', '20191221_gdeltdata.csv', '20191222_gdeltdata.csv', '20191223_gdeltdata.csv', '20191224_gdeltdata.csv', '20191225_gdeltdata.csv', '20191226_gdeltdata.csv', '20191227_gdeltdata.csv', '20191228_gdeltdata.csv']
['20191229_gdeltdata.csv', '20191230_gdeltdata.csv', '20191231_gdeltdata.csv', '20200101_gdeltdata.csv', '20200102_gdeltdata.csv', '20200103_gdeltdata.csv', '20200104_

/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

['20191127_gdeltdata.csv', '20191128_gdeltdata.csv', '20191129_gdeltdata.csv', '20191130_gdeltdata.csv', '20191202_gdeltdata.csv', '20191203_gdeltdata.csv', '20191204_gdeltdata.csv', '20191205_gdeltdata.csv', '20191206_gdeltdata.csv', '20191207_gdeltdata.csv', '20191208_gdeltdata.csv', '20191209_gdeltdata.csv', '20191210_gdeltdata.csv', '20191212_gdeltdata.csv', '20191213_gdeltdata.csv', '20191214_gdeltdata.csv', '20191215_gdeltdata.csv', '20191216_gdeltdata.csv', '20191217_gdeltdata.csv', '20191218_gdeltdata.csv', '20191219_gdeltdata.csv', '20191220_gdeltdata.csv', '20191221_gdeltdata.csv', '20191222_gdeltdata.csv', '20191223_gdeltdata.csv', '20191224_gdeltdata.csv', '20191225_gdeltdata.csv', '20191226_gdeltdata.csv', '20191227_gdeltdata.csv', '20191228_gdeltdata.csv']
['20191229_gdeltdata.csv', '20191230_gdeltdata.csv', '20191231_gdeltdata.csv', '20200101_gdeltdata.csv', '20200102_gdeltdata.csv', '20200103_gdeltdata.csv', '20200104_gdeltdata.csv', '20200105_gdeltdata.csv', '20200106_

## read downloaded files into RDDs

In [5]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

data_before = sqlContext.read.option("header", "true").csv(results_1_fix_bug)
dbg(test_data_before)
data_before_rdd = data_before.rdd
dbg(data_before_rdd)

data_after = sqlContext.read.option("header", "true").csv(results_2_fix_bug)
dbg(test_data_after)
data_after_rdd = data_after.rdd
dbg(data_after_rdd)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


['20191127_gdeltdata.csv', '20191128_gdeltdata.csv', '20191129_gdeltdata.csv', '20191130_gdeltdata.csv', '20191201_gdeltdata.csv', '20191202_gdeltdata.csv', '20191203_gdeltdata.csv', '20191204_gdeltdata.csv', '20191205_gdeltdata.csv', '20191206_gdeltdata.csv', '20191207_gdeltdata.csv', '20191208_gdeltdata.csv', '20191209_gdeltdata.csv', '20191210_gdeltdata.csv', '20191211_gdeltdata.csv', '20191212_gdeltdata.csv', '20191213_gdeltdata.csv', '20191214_gdeltdata.csv', '20191215_gdeltdata.csv', '20191216_gdeltdata.csv', '20191217_gdeltdata.csv', '20191218_gdeltdata.csv', '20191219_gdeltdata.csv', '20191220_gdeltdata.csv', '20191221_gdeltdata.csv', '20191222_gdeltdata.csv', '20191223_gdeltdata.csv', '20191224_gdeltdata.csv', '20191225_gdeltdata.csv', '20191226_gdeltdata.csv', '20191227_gdeltdata.csv', '20191228_gdeltdata.csv']
[('20191212234500-0', '20191212234500'), ('20191212234500-1', '20191212234500'), ('20191212234500-2', '20191212234500'), ('20191212234500-3', '20191212234500'), ('2019

In [6]:
data_before.show()
data_after.show()

+-----------------+--------------+--------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+
|      GKGRECORDID|          DATE|SourceCollectionIdentifier|    SourceCommonName|  DocumentIdentifier|              Counts|            V2Counts|              Themes|            V2Themes|           Locations|         V2Locations|             Persons|           V2Persons|       Organizations|     V2Organizations|              V2Tone|               Dates|                GCAM|        SharingImage|RelatedImages|   SocialImageEmbeds|   SocialVideo

In [7]:
# Allocate Locations and 2VTone as rdd 
import time
start_time = time.monotonic() 

print("="*20 + "Before"+ "="*20)
print("-"*20 + "Allocate Locations and 2VTone as rdd" + "-"*20)
Locations_V2Tone_before = data_before_rdd.map(lambda a: (a.Locations, a.V2Tone))
dbg(Locations_V2Tone_before)
print("-"*20 + "Allocate Locations and 2VThemes as rdd" + "-"*20)
Locations_V2Themes_before = data_before_rdd.map(lambda a: (a.Locations, a.V2Themes))
dbg(Locations_V2Themes_before)

print("="*20 + "After"+ "="*20)
print("-"*20 + "Allocate Locations and 2VTone as rdd" + "-"*20)
Locations_V2Tone_after = data_after_rdd.map(lambda a: (a.Locations, a.V2Tone))
dbg(Locations_V2Tone_after)
print("-"*20 + "Allocate Locations and 2VThemes as rdd" + "-"*20)
Locations_V2Themes_after = data_after_rdd.map(lambda a: (a.Locations, a.V2Themes))
dbg(Locations_V2Themes_after)

end_time = time.monotonic()
print("The time used to allocate Locations, 2VTone and 2VThemes as rdd ",end_time-start_time)

====================Before====================
--------------------Allocate Locations and 2VTone as rdd--------------------
[('1#Australia#AS#AS#-25#135#AS;1#United Kingdom#UK#UK#54#-4#UK', '4.43548387096774,5.37634408602151,0.940860215053763,6.31720430107527,20.9677419354839,2.55376344086022,679'), ('2#Florida, United States#US#USFL#27.8333#-81.717#FL;2#North Carolina, United States#US#USNC#35.6411#-79.8431#NC', '-4.15094339622642,3.0188679245283,7.16981132075472,10.188679245283,22.0754716981132,1.32075471698113,495'), (None, '-0.476190476190476,1.58730158730159,2.06349206349206,3.65079365079365,19.3650793650794,3.96825396825397,540'), ('2#New Jersey, United States#US#USNJ#40.314#-74.5089#NJ', '2.46305418719212,3.94088669950739,1.47783251231527,5.41871921182266,23.9737274220033,0.821018062397373,543'), ('4#Irish Sea, Ireland (General), Ireland#EI#EI00#53.6667#-5#-1503536;1#Ireland#EI#EI#53#-8#EI;1#United Kingdom#UK#UK#54#-4#UK;4#Brussels, Bruxelles-Capitale, Belgium#BE#BE11#50.8333#4.

In [8]:
import time
start_time = time.monotonic() 
#  Countrycode, Tone
def country_tone(string):
  if string != None:
    country_code = string.split('#')
    if len(country_code) >= 3:
      return country_code[2]
    else:
      return None

countrys = ['UK', 'US', 'NZ', 'FR', 'CA', 'CH']

Countrycode_V2Tone_before = (Locations_V2Tone_before
                  .map(lambda x: (country_tone(x[0]), x[1]))
                  .filter(lambda x: x[0] != None and x[0] in countrys)
                  .groupByKey()
                  .sortByKey())
dbg(Countrycode_V2Tone_before)

country_codes_before = [(x[0], len(x[1]))  for x in Countrycode_V2Tone_before.collect()]
dbg(country_codes_before)

Countrycode_V2Tone_after = (Locations_V2Tone_after
                  .map(lambda x: (country_tone(x[0]), x[1]))
                  .filter(lambda x: x[0] != None and x[0] in countrys)
                  .groupByKey()
                  .sortByKey())
dbg(Countrycode_V2Tone_after)

country_codes_after = [(x[0], len(x[1])) for x in Countrycode_V2Tone_after.collect()]
dbg(country_codes_after)

#  Countrycode, V2Themes
Country_V2Themes_before = (Locations_V2Themes_before
                  .map(lambda x: (country_tone(x[0]), x[1]))
                  .filter(lambda x: x[0] != None and x[0] in countrys))
dbg(Country_V2Themes_before)

Country_V2Themes_after = (Locations_V2Themes_after
                  .map(lambda x: (country_tone(x[0]), x[1]))
                  .filter(lambda x: x[0] != None and x[0] in countrys))
dbg(Country_V2Themes_after)

end_time = time.monotonic()
print("The time used to allocate Country, 2VTone and 2VThemes before and after Covid",end_time-start_time)

[('CA', ['-2.85714285714286,3.02521008403361,5.88235294117647,8.90756302521008,26.3865546218487,0.504201680672269,578', '-8.8339222614841,1.06007067137809,9.89399293286219,10.9540636042403,21.5547703180212,0.353356890459364,242', '-10.1226993865031,0.613496932515337,10.7361963190184,11.3496932515337,25.1533742331288,0,294', '-2.57510729613734,0.858369098712446,3.43347639484979,4.29184549356223,25.3218884120172,0,222', '-4.16666666666667,0,4.16666666666667,4.16666666666667,27.7777777777778,2.77777777777778,66', '5.0561797752809,7.30337078651685,2.24719101123596,9.55056179775281,14.0449438202247,1.12359550561798,141', '-6.58761528326746,1.71277997364954,8.300395256917,10.0131752305665,24.5059288537549,0.395256916996047,677', '3.37837837837838,4.27927927927928,0.900900900900901,5.18018018018018,11.036036036036,0,323', '-5.21739130434783,2.60869565217391,7.82608695652174,10.4347826086957,23.4782608695652,0,101', '0.664451827242525,3.15614617940199,2.49169435215947,5.64784053156146,23.58803

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[('CA', 2177), ('CH', 2081), ('FR', 273), ('NZ', 327), ('UK', 1926), ('US', 23512)]
[('US', 'GENERAL_GOVERNMENT,1389;GENERAL_GOVERNMENT,2643;EPU_POLICY_GOVERNMENT,1389;EPU_POLICY_GOVERNMENT,2643;TAX_POLITICAL_PARTY_REPUBLICANS,1463;TAX_POLITICAL_PARTY_REPUBLICANS,1894;UNGP_EDUCATION,562;MANMADE_DISASTER_IMPLIED,1097;SOC_POINTSOFINTEREST_SCHOOL,1097;WB_695_POVERTY,974;WB_701_JOBS_AND_POVERTY,974;WB_1170_JOB_CREATION_AND_JOB_OPPORTUNITIES,974;UNGP_JOB_OPPORTUNITIES_EMPLOYMENT,974;WB_678_DIGITAL_GOVERNMENT,3238;WB_694_BROADCAST_AND_MEDIA,3238;WB_133_INFORMATION_AND_COMMUNICATION_TECHNOLOGIES,3238;DELAY,1601;USPEC_UNCERTAINTY1,1601;ECON_TAXATION,1348;EPU_POLICY_TAX,1348;EPU_CATS_TAXES,1348;TAX_FNCACT_STUDENTS,271;TAX_FNCACT_STUDENTS,468;TAX_FNCACT_STUDENTS,734;TAX_FNCACT_STUDENTS,1797;TAX_FNCACT_STUDENTS,1946;TAX_FNCACT_STUDENTS,2259;TAX_FNCACT_STUDENT,165;TAX_FNCACT_STUDENT,556;TAX_FNCACT_STUDENT,3031;WB_1979_NATURAL_RESOURCE_MANAGEMENT,912;WB_435_AGRICULTURE_AND_FOOD_SECURITY,912;WB_1986

In [9]:
import time
start_time = time.monotonic() 

def tidy_data(data):
  result = []
  for string in data:
    floats = string.split(",")
    tone = 0
    for f in floats[:-2]:
      tone += float(f)
    result.append(tone)
  return result


Countrycode_V2Tone_before_t = Countrycode_V2Tone_before.map(lambda x: (x[0], int(sum(tidy_data(x[1]))/ len(tidy_data(x[1])))))
dbg(Countrycode_V2Tone_before_t)

# for country, data in Countrycode_V2Tone_before_t.collect():
#   print(country, len(data))


Countrycode_V2Tone_after_t = Countrycode_V2Tone_after.map(lambda x: (x[0], int(sum(tidy_data(x[1]))/ len(tidy_data(x[1])))))
dbg(Countrycode_V2Tone_after_t)

# for country, data in Countrycode_V2Tone_after_t.collect():
#   print(country, len(data))

end_time = time.monotonic()
print("The time used to caculate the sum of V2Tones before and after covid start",end_time-start_time)

[('CA', 33), ('CH', 33), ('FR', 35), ('NZ', 35), ('UK', 36), ('US', 34)]
[('CA', 33), ('CH', 32), ('FR', 33), ('NZ', 34), ('UK', 36), ('US', 34)]
The time used to caculate the sum of V2Tones before and after covid start 0.7822126479999838


In [10]:
import time
start_time = time.monotonic() 

def cosine_similarity(list_a,list_b):
  """a regular python function that computes cosine similarity between two lists of floats"""
  numerator = 0
  denom_l = 0
  denom_r = 0 
  for i in range(len(list_a)):
    numerator += list_a[i]*list_b[i]
    denom_l += list_a[i]**2
    denom_r += list_b[i]**2
  return numerator / (((denom_l)**(1/2))*((denom_r)**(1/2)))

before = [tone for country, tone in Countrycode_V2Tone_before_t.collect()]
after = [tone for country, tone in Countrycode_V2Tone_after_t.collect()]

cossim_before_after = cosine_similarity(before, after)
dbg(cossim_before_after)

end_time = time.monotonic()
print("The time used to caculate the cossine-similarity of V2Tone before and after covid start",end_time-start_time)

0.9997569330516283
The time used to caculate the cossine-similarity of V2Tone before and after covid start 0.6262270110000259


In [11]:
import time
start_time = time.monotonic() 

#convert theme from string to a list
def convert_list(data):
  result = []
  if data != None and data != '':
    themes = data.split(";")
    for s in themes:
      sl = s.split(",")
      for n in sl:
        if n != "" and not n.isdigit():
          result.append((n, 1))
      # result += themes
  return result

Country_V2Themes_before_r = Country_V2Themes_before.map(lambda x: (x[0], convert_list(x[1])))
dbg(Country_V2Themes_before_r)
Country_V2Themes_after_r = Country_V2Themes_after.map(lambda x: (x[0], convert_list(x[1])))
dbg(Country_V2Themes_after_r)

end_time = time.monotonic()
print("The time used to form countries and 2VThemes",end_time-start_time)

[('US', [('GENERAL_GOVERNMENT', 1), ('GENERAL_GOVERNMENT', 1), ('EPU_POLICY_GOVERNMENT', 1), ('EPU_POLICY_GOVERNMENT', 1), ('TAX_POLITICAL_PARTY_REPUBLICANS', 1), ('TAX_POLITICAL_PARTY_REPUBLICANS', 1), ('UNGP_EDUCATION', 1), ('MANMADE_DISASTER_IMPLIED', 1), ('SOC_POINTSOFINTEREST_SCHOOL', 1), ('WB_695_POVERTY', 1), ('WB_701_JOBS_AND_POVERTY', 1), ('WB_1170_JOB_CREATION_AND_JOB_OPPORTUNITIES', 1), ('UNGP_JOB_OPPORTUNITIES_EMPLOYMENT', 1), ('WB_678_DIGITAL_GOVERNMENT', 1), ('WB_694_BROADCAST_AND_MEDIA', 1), ('WB_133_INFORMATION_AND_COMMUNICATION_TECHNOLOGIES', 1), ('DELAY', 1), ('USPEC_UNCERTAINTY1', 1), ('ECON_TAXATION', 1), ('EPU_POLICY_TAX', 1), ('EPU_CATS_TAXES', 1), ('TAX_FNCACT_STUDENTS', 1), ('TAX_FNCACT_STUDENTS', 1), ('TAX_FNCACT_STUDENTS', 1), ('TAX_FNCACT_STUDENTS', 1), ('TAX_FNCACT_STUDENTS', 1), ('TAX_FNCACT_STUDENTS', 1), ('TAX_FNCACT_STUDENT', 1), ('TAX_FNCACT_STUDENT', 1), ('TAX_FNCACT_STUDENT', 1), ('WB_1979_NATURAL_RESOURCE_MANAGEMENT', 1), ('WB_435_AGRICULTURE_AND_FOO

In [16]:
import time
start_time = time.monotonic() 

# countrys = ['UK', 'US', 'NZ', 'FR', 'CA', 'CH']
min_frequency = 500

def frequentitems_baskets(country_code, dataset):
  if country_code == 'NZ' or country_code == 'FR':
    min_frequency = 50
  elif country_code == 'US':
    min_frequency = 5000
  else:
    min_frequency = 500 
  baskets = (dataset
             .filter(lambda x: x[0] == country_code)
             .map(lambda x: convert_baskets(x[1])))
  total = baskets.count()
  frequentitems = (sc
                    .parallelize(events_tidy(dataset
                                             .filter(lambda x: x[0] == country_code)
                                             .map(lambda x: x[1]).collect()))
                    .reduceByKey(lambda a, b: a+b)
                    .filter(lambda a: a[1] >= min_frequency and a[1] <= total))
  return frequentitems, frequentitems.collectAsMap(), baskets

def events_tidy(data):
  result = []
  for d in data:
    result += d
  return result

def convert_baskets(data):
  result = []
  for d in data:
    result.append(d[0])
  return result


print("|||=====UK=====|||")
print("--------------------------------Before---------------------------------")
Themes_UK_be_nf, Themes_UK_be_f, Themes_UK_be_baskets = frequentitems_baskets('UK', Country_V2Themes_before_r)
dbg(Themes_UK_be_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_UK_be_baskets)
# dbg(Themes_UK_be_baskets.count())
print("--------------------------------After---------------------------------")
Themes_UK_af_nf, Themes_UK_af_f, Themes_UK_af_baskets = frequentitems_baskets('UK', Country_V2Themes_after_r)
dbg(Themes_UK_af_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_UK_af_baskets)
# dbg(Themes_UK_af_baskets.count())
print("|||=====US=====|||")
print("--------------------------------Before---------------------------------")
Themes_US_be_nf, Themes_US_be_f, Themes_US_be_baskets = frequentitems_baskets('US', Country_V2Themes_before_r)
dbg(Themes_US_be_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_US_be_baskets)
print("--------------------------------After---------------------------------")
Themes_US_af_nf, Themes_US_af_f, Themes_US_af_baskets = frequentitems_baskets('US', Country_V2Themes_after_r)
dbg(Themes_US_af_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_US_af_baskets)
print("|||=====NZ=====|||")
print("--------------------------------Before---------------------------------")
Themes_NZ_be_nf, Themes_NZ_be_f, Themes_NZ_be_baskets = frequentitems_baskets('NZ', Country_V2Themes_before_r)
dbg(Themes_NZ_be_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_NZ_be_baskets)
print("--------------------------------After---------------------------------")
Themes_NZ_af_nf, Themes_NZ_af_f, Themes_NZ_af_baskets = frequentitems_baskets('NZ', Country_V2Themes_after_r)
dbg(Themes_NZ_af_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_NZ_af_baskets)
print("|||=====FR=====|||")
print("--------------------------------Before---------------------------------")
Themes_FR_be_nf, Themes_FR_be_f, Themes_FR_be_baskets = frequentitems_baskets('FR', Country_V2Themes_before_r)
dbg(Themes_FR_be_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_FR_be_baskets)
print("--------------------------------After---------------------------------")
Themes_FR_af_nf, Themes_FR_af_f, Themes_FR_af_baskets = frequentitems_baskets('FR', Country_V2Themes_after_r)
dbg(Themes_FR_af_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_FR_af_baskets)
print("|||=====CA=====|||")
print("--------------------------------Before---------------------------------")
Themes_CA_be_nf, Themes_CA_be_f, Themes_CA_be_baskets = frequentitems_baskets('CA', Country_V2Themes_before_r)
dbg(Themes_CA_be_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_CA_be_baskets)
print("--------------------------------After---------------------------------")
Themes_CA_af_nf, Themes_CA_af_f, Themes_CA_af_baskets = frequentitems_baskets('CA', Country_V2Themes_after_r)
dbg(Themes_CA_af_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_CA_af_baskets)
print("|||=====CH=====|||")
print("--------------------------------Before---------------------------------")
Themes_CH_be_nf, Themes_CH_be_f, Themes_CH_be_baskets = frequentitems_baskets('CH', Country_V2Themes_before_r)
dbg(Themes_CH_be_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_CH_be_baskets)
print("--------------------------------After---------------------------------")
Themes_CH_af_nf, Themes_CH_af_f, Themes_CH_af_baskets = frequentitems_baskets('CH', Country_V2Themes_after_r)
dbg(Themes_CH_af_nf.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_CH_af_baskets)

end_time = time.monotonic()
print("The time used to gets frequent items and baskets",end_time-start_time)

|||=====UK=====|||
--------------------------------Before---------------------------------
[('MANMADE_DISASTER_IMPLIED', 1369), ('SECURITY_SERVICES', 1360), ('CRISISLEX_C07_SAFETY', 1355), ('TAX_FNCACT_POLICE', 1347), ('TAX_FNCACT_MINISTER', 1323)]
--------------------------------After---------------------------------
[('USPEC_POLITICS_GENERAL1', 1244), ('CRISISLEX_C07_SAFETY', 1124), ('CRISISLEX_CRISISLEXREC', 1087), ('TAX_FNCACT_POLICE', 1038), ('SECURITY_SERVICES', 1037)]
|||=====US=====|||
--------------------------------Before---------------------------------
[('LEADER', 16385), ('USPEC_POLITICS_GENERAL1', 15787), ('TAX_FNCACT_PRESIDENT', 13583), ('CRISISLEX_C07_SAFETY', 13084), ('EDUCATION', 11846)]
--------------------------------After---------------------------------
[('LEADER', 20888), ('USPEC_POLITICS_GENERAL1', 20771), ('TAX_FNCACT_PRESIDENT', 16582), ('CRISISLEX_C07_SAFETY', 15970), ('CRISISLEX_CRISISLEXREC', 14732)]
|||=====NZ=====|||
--------------------------------Before

In [17]:
import time
start_time = time.monotonic() 

def combinate(broadcast_value , line):
  ans = []
  for i, word1 in enumerate(line):
    if word1 in broadcast_value:
      for j, word2 in enumerate(line):
        if i > j and word2 in broadcast_value:
          ans.append(tuple(sorted([word1, word2])))
  return ans

def apriori(frequentitems, baskets):
  broadcast_value = sc.broadcast(frequentitems).value
  themes_groups = (baskets
                   .flatMap(lambda x: combinate(broadcast_value, x))
                   .map(lambda a:(a, 1))
                   .reduceByKey(lambda a,b:a+b))
  return themes_groups

def support(group_rdd, baskets):
  total = baskets.count()
  return group_rdd.map(lambda x: (x[0],x[1]/total))

print("|||=====UK=====|||")
print("--------------------------------Before---------------------------------")
Themes_UK_before_b = apriori(Themes_UK_be_f, Themes_UK_be_baskets)
Themes_UK_before_support = support(Themes_UK_be_nf, Themes_UK_be_baskets)
dbg(Themes_UK_before_support.takeOrdered(5, lambda s: -1 * s[1]))
# dbg(Themes_UK_be_f)
# dbg(Themes_UK_before_b)
print("--------------------------------After---------------------------------")
Themes_UK_after_b = apriori(Themes_UK_af_f, Themes_UK_af_baskets)
Themes_UK_after_support = support(Themes_UK_af_nf, Themes_UK_af_baskets)
dbg(Themes_UK_after_support.takeOrdered(5, lambda s: -1 * s[1]))
print("|||=====US=====|||")
print("--------------------------------Before---------------------------------")
Themes_US_before_b = apriori(Themes_US_be_f, Themes_US_be_baskets)
Themes_US_before_support = support(Themes_US_be_nf, Themes_US_be_baskets)
dbg(Themes_US_before_support.takeOrdered(5, lambda s: -1 * s[1]))
print("--------------------------------After---------------------------------")
Themes_US_after_b = apriori(Themes_US_af_f, Themes_US_af_baskets)
Themes_US_after_support = support(Themes_US_af_nf, Themes_US_af_baskets)
dbg(Themes_US_after_support.takeOrdered(5, lambda s: -1 * s[1]))
print("|||=====NZ=====|||")
print("--------------------------------Before---------------------------------")
Themes_NZ_before_b = apriori(Themes_NZ_be_f, Themes_NZ_be_baskets)
Themes_NZ_before_support = support(Themes_NZ_be_nf, Themes_NZ_be_baskets)
dbg(Themes_NZ_before_support.takeOrdered(5, lambda s: -1 * s[1]))
print("--------------------------------After---------------------------------")
Themes_NZ_after_b = apriori(Themes_NZ_af_f, Themes_NZ_af_baskets)
Themes_NZ_after_support = support(Themes_NZ_af_nf, Themes_NZ_af_baskets)
dbg(Themes_NZ_after_support.takeOrdered(5, lambda s: -1 * s[1]))
print("|||=====FR=====|||")
print("--------------------------------Before---------------------------------")
Themes_FR_before_b = apriori(Themes_FR_be_f, Themes_FR_be_baskets)
Themes_FR_before_support = support(Themes_FR_be_nf, Themes_FR_be_baskets)
dbg(Themes_FR_before_support.takeOrdered(5, lambda s: -1 * s[1]))
print("--------------------------------After---------------------------------")
Themes_FR_after_b = apriori(Themes_FR_af_f, Themes_FR_af_baskets)
Themes_FR_after_support = support(Themes_FR_af_nf, Themes_FR_af_baskets)
dbg(Themes_FR_after_support.takeOrdered(5, lambda s: -1 * s[1]))
print("|||=====CA=====|||")
print("--------------------------------Before---------------------------------")
Themes_CA_before_b = apriori(Themes_CA_be_f, Themes_CA_be_baskets)
Themes_CA_before_support = support(Themes_CA_be_nf, Themes_CA_be_baskets)
dbg(Themes_CA_before_support.takeOrdered(5, lambda s: -1 * s[1]))
print("--------------------------------After---------------------------------")
Themes_CA_after_b = apriori(Themes_CA_af_f, Themes_CA_af_baskets)
Themes_CA_after_support = support(Themes_CA_af_nf, Themes_CA_af_baskets)
dbg(Themes_CA_after_support.takeOrdered(5, lambda s: -1 * s[1]))
print("|||=====CH=====|||")
print("--------------------------------Before---------------------------------")
Themes_CH_before_b = apriori(Themes_CH_be_f, Themes_CH_be_baskets)
Themes_CH_before_support = support(Themes_CH_be_nf, Themes_CH_be_baskets)
dbg(Themes_CH_before_support.takeOrdered(5, lambda s: -1 * s[1]))
print("--------------------------------After---------------------------------")
Themes_CH_after_b = apriori(Themes_CH_af_f, Themes_CH_af_baskets)
Themes_CH_after_support = support(Themes_CH_af_nf, Themes_CH_af_baskets)
dbg(Themes_CH_after_support.takeOrdered(5, lambda s: -1 * s[1]))

end_time = time.monotonic()
print("The time used to caculate supports and print the top5 results",end_time-start_time)

|||=====UK=====|||
--------------------------------Before---------------------------------
[('MANMADE_DISASTER_IMPLIED', 0.6998977505112475), ('SECURITY_SERVICES', 0.6952965235173824), ('CRISISLEX_C07_SAFETY', 0.6927402862985685), ('TAX_FNCACT_POLICE', 0.6886503067484663), ('TAX_FNCACT_MINISTER', 0.6763803680981595)]
--------------------------------After---------------------------------
[('USPEC_POLITICS_GENERAL1', 0.6458982346832814), ('CRISISLEX_C07_SAFETY', 0.5835929387331257), ('CRISISLEX_CRISISLEXREC', 0.5643821391484943), ('TAX_FNCACT_POLICE', 0.5389408099688473), ('SECURITY_SERVICES', 0.5384215991692627)]
|||=====US=====|||
--------------------------------Before---------------------------------
[('LEADER', 0.8521427085500312), ('USPEC_POLITICS_GENERAL1', 0.8210422300811316), ('TAX_FNCACT_PRESIDENT', 0.7064177241522779), ('CRISISLEX_C07_SAFETY', 0.6804659871021427), ('EDUCATION', 0.6160807156230497)]
--------------------------------After---------------------------------
[('LEADER

In [14]:
import time
start_time = time.monotonic() 

# countrys = ['UK', 'US', 'NZ', 'FR', 'CA', 'CH']
def confidence_interest(individual_pairs, Frequent_Themes, baskets):
  # interest = |confidence - Pr(y)|
  # Pr(y) = y_counts/total_baskets
  count_of_total_pairs = baskets.count()
  if count_of_total_pairs <= 500:
    proportion = 100
  elif count_of_total_pairs <= 5000:
    proportion = 20
  else:
    proportion = 10
  sup_individual = individual_pairs.map(lambda a:(a[0], a[1]/count_of_total_pairs))
  sup_ij = Frequent_Themes.map(lambda a:(a[0][0], (a[0][1], a[1]/count_of_total_pairs)))
  sup_ji = Frequent_Themes.map(lambda a:(a[0][1], (a[0][0], a[1]/count_of_total_pairs)))
  sup_all_ij_and_ji = sup_ij.union(sup_ji)
  all = sup_all_ij_and_ji.join(sup_individual)
  confidence = all.map(lambda a: ((a[0], a[1][0][0]), a[1][0][1]/a[1][1]/proportion))
  interest = confidence.map(lambda x: (x[0][1], x)).join(sup_individual).map(lambda x: (x[1][0][0], x[1][0][1] - x[1][1]))
  return confidence, interest

print("|||=====UK=====|||")
print("--------------------------------Before---------------------------------")
confidence_UK_be, interest_UK_be = confidence_interest(Themes_UK_be_nf, Themes_UK_before_b, Themes_UK_be_baskets)
dbg(confidence_UK_be.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_UK_be.takeOrdered(5, lambda s: -1 * s[1]))
print("--------------------------------After---------------------------------")
confidence_UK_af, interest_UK_af = confidence_interest(Themes_UK_af_nf, Themes_UK_after_b, Themes_UK_af_baskets)
dbg(confidence_UK_af.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_UK_af.takeOrdered(5, lambda s: -1 * s[1]))
print("|||=====US=====|||")
print("--------------------------------Before---------------------------------")
confidence_US_be, interest_US_be = confidence_interest(Themes_US_be_nf, Themes_US_before_b, Themes_US_be_baskets)
dbg(confidence_US_be.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_US_be.takeOrdered(5, lambda s: -1 * s[1]))
print("--------------------------------After---------------------------------")
confidence_US_af, interest_US_af = confidence_interest(Themes_US_af_nf, Themes_US_after_b, Themes_US_af_baskets)
dbg(confidence_US_af.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_US_af.takeOrdered(5, lambda s: -1 * s[1]))
print("|||=====NZ=====|||")
print("--------------------------------Before---------------------------------")
confidence_NZ_be, interest_NZ_be = confidence_interest(Themes_NZ_be_nf, Themes_NZ_before_b, Themes_NZ_be_baskets)
dbg(confidence_NZ_be.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_NZ_be.takeOrdered(5, lambda s: -1 * s[1]))
print("--------------------------------After---------------------------------")
confidence_NZ_af, interest_NZ_af = confidence_interest(Themes_NZ_af_nf, Themes_NZ_after_b, Themes_NZ_af_baskets)
dbg(confidence_NZ_af.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_NZ_af.takeOrdered(5, lambda s: -1 * s[1]))
print("|||=====FR=====|||")
print("--------------------------------Before---------------------------------")
confidence_FR_be, interest_FR_be = confidence_interest(Themes_FR_be_nf, Themes_FR_before_b, Themes_FR_be_baskets)
dbg(confidence_FR_be.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_FR_be.takeOrdered(5, lambda s: -1 * s[1]))
print("--------------------------------After---------------------------------")
confidence_FR_af, interest_FR_af = confidence_interest(Themes_FR_af_nf, Themes_FR_after_b, Themes_FR_af_baskets)
dbg(confidence_FR_af.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_FR_af.takeOrdered(5, lambda s: -1 * s[1]))
print("|||=====CA=====|||")
print("--------------------------------Before---------------------------------")
confidence_CA_be, interest_CA_be = confidence_interest(Themes_CA_be_nf, Themes_CA_before_b, Themes_CA_be_baskets)
dbg(confidence_CA_be.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_CA_be.takeOrdered(5, lambda s: -1 * s[1]))
print("--------------------------------After---------------------------------")
confidence_CA_af, interest_CA_af = confidence_interest(Themes_CA_af_nf, Themes_CA_after_b, Themes_CA_af_baskets)
dbg(confidence_CA_af.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_CA_af.takeOrdered(5, lambda s: -1 * s[1]))
print("|||=====CH=====|||")
print("--------------------------------Before---------------------------------")
confidence_CH_be, interest_CH_be = confidence_interest(Themes_CH_be_nf, Themes_CH_before_b, Themes_CH_be_baskets)
dbg(confidence_CH_be.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_CH_be.takeOrdered(5, lambda s: -1 * s[1]))
print("--------------------------------After---------------------------------")
confidence_CH_af, interest_CH_af = confidence_interest(Themes_CH_af_nf, Themes_CH_after_b, Themes_CH_af_baskets)
dbg(confidence_CH_af.takeOrdered(5, lambda s: -1 * s[1]))
dbg(interest_CH_af.takeOrdered(5, lambda s: -1 * s[1]))

end_time = time.monotonic()
print("The time used to caculate confidences and intrests then print the top5 results",end_time-start_time)

|||=====UK=====|||
--------------------------------Before---------------------------------
[(('MARITIME_INCIDENT', 'MANMADE_DISASTER_IMPLIED'), 0.4611111111111111), (('MARITIME_INCIDENT', 'MARITIME'), 0.4598290598290598), (('TAX_FNCACT_POLICE', 'SECURITY_SERVICES'), 0.44539717891610986), (('MARITIME', 'MANMADE_DISASTER_IMPLIED'), 0.4421311475409836), (('SECURITY_SERVICES', 'TAX_FNCACT_POLICE'), 0.441139705882353)]
[(('MARITIME_INCIDENT', 'MARITIME'), 0.1479681191337633), (('MARITIME', 'MARITIME_INCIDENT'), 0.14190385195615007), (('MARITIME_INCIDENT', 'MARITIME_INCIDENT'), -0.094122489643962), (('MARITIME_INCIDENT', 'MARITIME_INCIDENT'), -0.094122489643962), (('MANMADE_DISASTER_IMPLIED', 'MARITIME_INCIDENT'), -0.10203811837040158)]
--------------------------------After---------------------------------
[(('SECURITY_SERVICES', 'TAX_FNCACT_POLICE'), 0.3158148505303761), (('TAX_FNCACT_POLICE', 'SECURITY_SERVICES'), 0.31551059730250486), (('SECURITY_SERVICES', 'CRISISLEX_C07_SAFETY'), 0.2728

In [15]:
!zip -o mydata.zip *.csv
# !zip -o articles.zip articles/*
# !unzip -o articles.zip

  adding: 20191127_gdeltdata.csv (deflated 68%)
  adding: 20191128_gdeltdata.csv (deflated 68%)
  adding: 20191129_gdeltdata.csv (deflated 68%)
  adding: 20191130_gdeltdata.csv (deflated 68%)
  adding: 20191202_gdeltdata.csv (deflated 68%)
  adding: 20191203_gdeltdata.csv (deflated 68%)
  adding: 20191204_gdeltdata.csv (deflated 68%)
  adding: 20191205_gdeltdata.csv (deflated 68%)
  adding: 20191206_gdeltdata.csv (deflated 68%)
  adding: 20191207_gdeltdata.csv (deflated 68%)
  adding: 20191208_gdeltdata.csv (deflated 68%)
  adding: 20191209_gdeltdata.csv (deflated 68%)
  adding: 20191210_gdeltdata.csv (deflated 68%)
  adding: 20191212_gdeltdata.csv (deflated 68%)
  adding: 20191213_gdeltdata.csv (deflated 68%)
  adding: 20191214_gdeltdata.csv (deflated 68%)
  adding: 20191215_gdeltdata.csv (deflated 68%)
  adding: 20191216_gdeltdata.csv (deflated 68%)
  adding: 20191217_gdeltdata.csv (deflated 68%)
  adding: 20191218_gdeltdata.csv (deflated 68%)
  adding: 20191219_gdeltdata.csv (deflat